In [40]:
pip install matplotlib tensorflow_addons

     |████████████████████████████████| 679 kB 6.2 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [41]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization, Lambda, LeakyReLU
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input
# import tf_slim as slim

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageChops, ImageEnhance
import os
import itertools
import random
%matplotlib inline
np.random.seed(2)

In [2]:
def initializer():
    filter1 = [[0, 0, 0, 0, 0],
               [0, -1, 2, -1, 0],
               [0, 2, -4, 2, 0],
               [0, -1, 2, -1, 0],
               [0, 0, 0, 0, 0]]
    filter2 = [[-1, 2, -2, 2, -1],
               [2, -6, 8, -6, 2],
               [-2, 8, -12, 8, -2],
               [2, -6, 8, -6, 2],
               [-1, 2, -2, 2, -1]]
    filter3 = [[0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0],
               [0, 1, -2, 1, 0],
               [0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0]]
    q = [4.0, 12.0, 2.0]
    filter1 = np.asarray(filter1, dtype=float) / 4
    filter2 = np.asarray(filter2, dtype=float) / 12
    filter3 = np.asarray(filter3, dtype=float) / 2
    filters = [[filter1, filter1, filter1], [filter2, filter2, filter2], [filter3, filter3, filter3]]
    filters = np.einsum('klij->ijlk', filters)
    filters = tf.Variable(filters, dtype=tf.float32)
    return filters

In [3]:
def convert_to_ela_image(path, quality):
    temp_filename = 'temp_file_name.jpg'
    ela_filename = 'temp_ela.png'
    
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)
    
    ela_image = ImageChops.difference(image, temp_image)
    
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    
    return ela_image

In [4]:
h = 224
w = 224
image_size = (h, w)

# np_arr = lambda img: img.resize(image_size).flatten() / 255.0
np_arr = lambda img: np.array(img.resize(image_size)).flatten() / 255.0

def prepare_image(image_path, is_ela):
    return np_arr(convert_to_ela_image(image_path, 95)) if is_ela == 1 else np_arr(Image.open(image_path))

In [5]:
x_srm = [] # SRM converted images
x_ela = [] # ELA converted images
labels = [] # 0 for fake, 1 for real

In [6]:
def prepare_data(path, cls, srm, ela, targets):
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith('jpg') or filename.endswith('png'):
                try:
                    full_path = os.path.join(dirname, filename)
                    srm.append(prepare_image(full_path, 0))
                    ela.append(prepare_image(full_path, 1))
                    targets.append(cls)
                except:
                    pass
                if len(targets) % 500 == 0:
                    print('Processing {} images'.format(len(targets)))
    print(len(srm), len(ela), len(labels))

In [8]:
#place authentic
Au_path = '../synthetic/Au'
prepare_data(Au_path, 1, x_srm, x_ela, labels)

Processing 500 images
Processing 1000 images
Processing 1500 images
Processing 2000 images
Processing 2500 images
Processing 3000 images
Processing 3500 images
Processing 4000 images
Processing 4500 images
Processing 5000 images
Processing 5500 images
Processing 6000 images
Processing 6500 images
Processing 7000 images
Processing 7500 images
Processing 8000 images
8173 8173 8173


In [9]:
#place tampered
Tp_path = '../synthetic/Tp'
prepare_data(Tp_path, 0, x_srm, x_ela, labels)

Processing 8500 images
Processing 9000 images
Processing 9500 images
Processing 10000 images
Processing 10500 images
Processing 11000 images
Processing 11500 images
Processing 12000 images
12477 12477 12477


In [7]:
def reshaping(srm, ela, targets):
    srm = np.array(srm)
    ela = np.array(ela)

    targets = to_categorical(targets, 2)

    srm = srm.reshape(-1, h, w, 3)
    ela = ela.reshape(-1, h, w, 3)

    print(srm.shape, ela.shape, targets.shape)
    
    # stack so we can split on the same pair of images
    x_combined = np.stack((srm, ela), axis=4)

    x_train, x_val, y_train, y_val = train_test_split(x_combined, targets, test_size = 0.2, random_state=5)

    # take them apart
    x_train_srm = x_train[:,:,:,:,0]
    x_val_srm = x_val[:,:,:,:,0]

    x_train_ela = x_train[:,:,:,:,1]
    x_val_ela = x_val[:,:,:,:,1]
    
    return x_train_srm, x_val_srm, x_train_ela, x_val_ela, y_train, y_val


In [11]:
x_train_srm, x_val_srm, x_train_ela, x_val_ela, y_train, y_val = reshaping(x_srm, x_ela, labels)

(12477, 224, 224, 3) (12477, 224, 224, 3) (12477, 2)


In [8]:
initializer_srm = initializer()

In [13]:
def outer_product(x):
    #Einstein Notation  [batch,1,1,depth] x [batch,1,1,depth] -> [batch,depth,depth]
    phi_I = tf.einsum('ijkm,ijkn->imn',x[0],x[1])
    
    # Reshape from [batch_size,depth,depth] to [batch_size, depth*depth]
    phi_I = tf.reshape(phi_I,[-1,x[0].shape[3]*x[1].shape[3]])
    
    # Divide by feature map size [sizexsize]
    size1 = int(x[1].shape[1])
    size2 = int(x[1].shape[2])
    phi_I = tf.divide(phi_I, size1*size2)
    
    # Take signed square root of phi_I
    y_ssqrt = tf.multiply(tf.sign(phi_I),tf.sqrt(tf.abs(phi_I)+1e-12))
    
    # Apply l2 normalization
    z_l2 = tf.math.l2_normalize(y_ssqrt, axis=1)
    return z_l2

In [14]:
def conv_layers(input):
    x = Conv2D(32, 3, padding='valid', activation='relu')(input)
    x = Conv2D(32, 3, padding='valid', activation='relu')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv2D(64, 3, padding='valid', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, 3, padding='valid', activation='relu')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv2D(128, 5, padding='valid', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, 5, padding='valid', activation='relu')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = BatchNormalization()(x)

#     x = Conv2D(256, 5, padding='valid', activation='relu')(x)
#     x = BatchNormalization()(x)
#     x = Conv2D(256, 5, padding='valid', activation='relu')(x)
#     x = MaxPool2D(pool_size=2)(x)
#     x = BatchNormalization()(x)

#   x = Conv2D(512, 3, padding='valid', activation='relu')(x)
#   x = BatchNormalization()(x)
#   x = Conv2D(512, 3, padding='valid', activation='relu')(x)
#   x = MaxPool2D(pool_size=2)(x)
#   model = BatchNormalization()(x)

    return x

In [15]:
def conv_layers_2(input):
    x = Conv2D(32, 3, padding='same')(input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv2D(32, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = MaxPool2D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv2D(64, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = MaxPool2D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv2D(128, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = MaxPool2D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv2D(256, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, 3, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = MaxPool2D(pool_size=2)(x)
    x = BatchNormalization()(x)

    return x

In [16]:
# SRM stream
srm_input = Input(shape=[h, w, 3], name='srm_input')
op = tf.nn.conv2d(srm_input, initializer_srm, strides=[1, 1, 1, 1], padding='SAME', name='srm-layer')
srm_model = conv_layers(op)

# ELA stream
ela_input = Input(shape=[h, w, 3], name='ela_input')
ela_model = conv_layers(ela_input)

# Concatenate streams
# concat = tf.keras.layers.Concatenate()([srm_model, ela_model])

# Bilinear fusion
x = Lambda(outer_product, name='outer_product')([srm_model,ela_model])

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
# x = Dense(256)(x)
# x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization()(x)

output = Dense(2, activation='softmax')(x)

model = tf.keras.Model(inputs=[srm_input, ela_input], outputs=[output])
model.summary()

[2021-05-26 21:52:13.405 tensorflow-2-3-gp-ml-g4dn-12xlarge-1be1fa3f01e6a5645cff1d084b39:51 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-05-26 21:52:13.430 tensorflow-2-3-gp-ml-g4dn-12xlarge-1be1fa3f01e6a5645cff1d084b39:51 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
srm_input (InputLayer)          [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_srm-layer (TensorFl [(None, 224, 224, 3) 0           srm_input[0][0]                  
__________________________________________________________________________________________________
ela_input (InputLayer

In [32]:
epochs = 50
batch_size = 32
init_lr = 1e-4
# optimizer = Adam(lr = init_lr)
optimizer = Adam(learning_rate = init_lr, amsgrad=True)
# decay = init_lr/epochs
# optimizer = RMSprop(learning_rate = init_lr, centered = True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0,patience=5, verbose=0, mode='auto')

checkpoint_filepath = 'two_stream/checkpoint'
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_accuracy', mode='max', save_best_only=True)

# model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit([x_train_srm, x_train_ela], y_train, batch_size=batch_size, epochs=epochs, validation_data=([x_val_srm, x_val_ela], y_val), callbacks=[early_stopping, model_checkpoint_callback, reduce_lr])

Epoch 1/50
  2/312 [..............................] - ETA: 38s - loss: 0.6353 - accuracy: 0.7656WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0817s vs `on_train_batch_end` time: 0.1692s). Check your callbacks.


312/312 [==============================] - ETA: 0s - loss: 0.3885 - accuracy: 0.8557WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0123s vs `on_test_batch_end` time: 0.0711s). Check your callbacks.


312/312 [==============================] - 92s 294ms/step - loss: 0.3885 - accuracy: 0.8557 - val_loss: 0.6846 - val_accuracy: 0.5469
Epoch 2/50
312/312 [==============================] - 87s 280ms/step - loss: 0.3680 - accuracy: 0.8698 - val_loss: 0.4188 - val_accuracy: 0.8325
Epoch 3/50
312/312 [==============================] - 87s 279ms/step - loss: 0.3501 - accuracy: 0.8774 - val_loss: 0.3469 - val_accuracy: 0.8730
Epoch 4/50
312/312 [==============================] - 88s 282ms/step - loss: 0.3522 - accuracy: 0.8758 - val_loss: 0.3428 - val_accuracy: 0.8762
Epoch 5/50
312/312 [==============================] - 89s 284ms/step - loss: 0.3442 - accuracy: 0.8776 - val_loss: 0.3363 - val_accuracy: 0.8822
Epoch 6/50
312/312 [==============================] - 84s 270ms/step - loss: 0.3483 - accuracy: 0.8769 - val_loss: 0.3578 - val_accuracy: 0.8746
Epoch 7/50
312/312 [==============================] - 85s 272ms/step - loss: 0.3424 - accuracy: 0.8789 - val_loss: 0.3428 - val_accuracy: 0.8

In [24]:
model.save('two_stream.h5')

# Testing

In [59]:
x_srm_test = [] # SRM converted images
x_ela_test = [] # ELA converted images
test_labels = [] # 0 for fake, 1 for real

In [60]:
#place authentic
Au_path_test = '../synthetic_test/Au'
prepare_data(Au_path_test, 1, x_srm_test, x_ela_test, test_labels)
# random.shuffle(X)
print(len(x_srm_test), len(x_ela_test), len(test_labels))

Processing 500 images
790 790 790


In [61]:
#place tampered
Tp_path_test = '../synthetic_test/Tp'
prepare_data(Tp_path_test, 0, x_srm_test, x_ela_test, test_labels)
print(len(x_srm_test), len(x_ela_test), len(test_labels))

Processing 1000 images
1190 1190 1190


In [64]:
x_test_srm, x_test2_srm, x_test_ela, x_test2_ela, y_test, y_test2 = reshaping(x_srm_test, x_ela_test, test_labels)

(1190, 224, 224, 3) (1190, 224, 224, 3) (1190, 2)


In [65]:
from sklearn.metrics import average_precision_score

predictions = model.predict([x_test_srm, x_test_ela])
average_precision = average_precision_score(y_test, predictions)

In [66]:
average_precision

0.9113130895250358

In [67]:
model.evaluate([x_val_srm, x_val_ela], y_val)

8/8 [==============================] - 1s 158ms/step - loss: 0.2513 - accuracy: 0.9118


[0.2513410449028015, 0.9117646813392639]

# CASIA

In [9]:
casia_srm = [] # SRM converted images
casia_ela = [] # ELA converted images
casia_labels = [] # 0 for fake, 1 for real

In [10]:
#place authentic
Au_path_casia = '../forgery/data/Au'
prepare_data(Au_path_casia, 1, casia_srm, casia_ela, casia_labels)
# random.shuffle(X)
print(len(casia_srm), len(casia_ela), len(casia_labels))

Processing 500 images
Processing 1000 images
Processing 1500 images
Processing 2000 images
Processing 2500 images
Processing 3000 images
Processing 3500 images
Processing 4000 images
Processing 4500 images
Processing 5000 images
Processing 5500 images
Processing 6000 images
Processing 6500 images
Processing 7000 images
7329 7329 0
7329 7329 7329


In [11]:
#place tampered
Tp_path_casia = '../forgery/data/Tp'
prepare_data(Tp_path_casia, 0, casia_srm, casia_ela, casia_labels)
print(len(casia_srm), len(casia_ela), len(casia_labels))

Processing 7500 images
Processing 8000 images
Processing 8500 images
Processing 9000 images
9393 9393 0
9393 9393 9393


In [12]:
x_train_casia_srm, x_val_casia_srm, x_train_casia_ela, x_val_casia_ela, y_casia_train, y_casia_val = reshaping(casia_srm, casia_ela, casia_labels)

(9393, 224, 224, 3) (9393, 224, 224, 3) (9393, 2)


In [29]:
casia_model = tf.keras.models.load_model('two_stream.h5')

# for layer in casia_model.layers[:-1]:
#     layer.trainable = False

In [42]:
casia_model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC(), tfa.metrics.F1Score(num_classes=2, average="micro")])

In [ ]:
history = casia_model.fit([x_train_casia_srm, x_train_casia_ela], y_casia_train, batch_size=batch_size, epochs=epochs, validation_data=([x_val_casia_srm, x_val_casia_ela], y_casia_val), 
                    callbacks=[early_stopping, model_checkpoint_callback, reduce_lr])

Epoch 1/50
  2/235 [..............................] - ETA: 28s - loss: 0.0278 - accuracy: 1.0000 - auc_1: 1.0000 - f1_score: 1.0000WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0789s vs `on_train_batch_end` time: 0.1641s). Check your callbacks.


235/235 [==============================] - 68s 290ms/step - loss: 0.0711 - accuracy: 0.9687 - auc_1: 0.9969 - f1_score: 0.9687 - val_loss: 0.0888 - val_accuracy: 0.9617 - val_auc_1: 0.9946 - val_f1_score: 0.9617
Epoch 2/50
235/235 [==============================] - 66s 280ms/step - loss: 0.0701 - accuracy: 0.9703 - auc_1: 0.9970 - f1_score: 0.9703 - val_loss: 0.0930 - val_accuracy: 0.9638 - val_auc_1: 0.9943 - val_f1_score: 0.9638
Epoch 3/50
235/235 [==============================] - 63s 267ms/step - loss: 0.0698 - accuracy: 0.9697 - auc_1: 0.9971 - f1_score: 0.9697 - val_loss: 0.1046 - val_accuracy: 0.9585 - val_auc_1: 0.9934 - val_f1_score: 0.9585
Epoch 4/50
235/235 [==============================] - 61s 260ms/step - loss: 0.0620 - accuracy: 0.9738 - auc_1: 0.9977 - f1_score: 0.9738 - val_loss: 0.0965 - val_accuracy: 0.9585 - val_auc_1: 0.9943 - val_f1_score: 0.9585
Epoch 5/50
235/235 [==============================] - 61s 260ms/step - loss: 0.0660 - accuracy: 0.9707 - auc_1: 0.9970 

In [38]:
casia_model.save('casia_model.h5')